<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [1]:
##### Your Code Here #####
import pandas as pd

df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
df = df.drop(columns={'customerID'})
print(df.shape)
#print(df.dtypes)
df.head()

(7043, 20)


gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  Female              0     Yes         No       1           No   
1    Male              0      No         No      34          Yes   
2    Male              0      No         No       2          Yes   
3    Male              0      No         No      45           No   
4  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity OnlineBackup  \
0  No phone service             DSL             No          Yes   
1                No             DSL            Yes           No   
2                No             DSL            Yes          Yes   
3  No phone service             DSL            Yes           No   
4                No     Fiber optic             No           No   

  DeviceProtection TechSupport StreamingTV StreamingMovies        Contract  \
0               No          No          No              No  Month-to-month   
1              Yes          No          No              No        One year   
2               No          No          No              No  Month-to-month   
3              Yes         Yes          No              No        One year   
4               No          No          No              No  Month-to-month   

  PaperlessBilling              PaymentMethod  MonthlyCharges TotalCharges  \
0              Yes           Electronic check           29.85        29.85   
1               No               Mailed check           56.95       1889.5   
2              Yes               Mailed check           53.85       108.15   
3               No  Bank transfer (automatic)           42.30      1840.75   
4              Yes           Electronic check           70.70       151.65   

  Churn  
0    No  
1    No  
2   Yes  
3    No  
4   Yes

In [2]:
df['Churn'].value_counts()

No     5174
Yes    1869
Name: Churn, dtype: int64

In [3]:
clean = {
    'gender': {'female':0, 'male':1, 'Female':0, 'Male':1},
    'Partner': {'No':0, 'Yes':1},
    'Dependents': {'No':0, 'Yes':1},
    'PhoneService': {'No':0, 'Yes':1},
    'MultipleLines': {'No':0, 'Yes':1, 'No phone service': -1},
    'InternetService': {'DSL':0, 'Fiber optic':1, 'No':-1},
    'OnlineSecurity': {'No':0, 'Yes':1, 'No internet service':-1},
    'OnlineBackup': {'No':0, 'Yes':1, 'No internet service':-1},
    'DeviceProtection': {'No':0, 'Yes':1, 'No internet service':-1},
    'TechSupport': {'No':0, 'Yes':1, 'No internet service':-1},
    'StreamingTV': {'No':0, 'Yes':1, 'No internet service':-1},
    'StreamingMovies': {'No':0, 'Yes':1, 'No internet service':-1},
    'Contract': {'Month-to-month':0, 'One year':1, 'Two year': 2},
    'PaperlessBilling': {'No':0, 'Yes':1},
    'PaymentMethod': {'Electronic check': 0, 'Mailed check':1, 'Bank transfer (automatic)':2, 'Credit card (automatic)':3},
    'Churn': {'No':0, 'Yes':1}
}

In [4]:
df.replace(clean, inplace=True)
df.head()

gender  SeniorCitizen  Partner  Dependents  tenure  PhoneService  \
0       0              0        1           0       1             0   
1       1              0        0           0      34             1   
2       1              0        0           0       2             1   
3       1              0        0           0      45             0   
4       0              0        0           0       2             1   

   MultipleLines  InternetService  OnlineSecurity  OnlineBackup  \
0             -1                0               0             1   
1              0                0               1             0   
2              0                0               1             1   
3             -1                0               1             0   
4              0                1               0             0   

   DeviceProtection  TechSupport  StreamingTV  StreamingMovies  Contract  \
0                 0            0            0                0         0   
1                 1            0            0                0         1   
2                 0            0            0                0         0   
3                 1            1            0                0         1   
4                 0            0            0                0         0   

   PaperlessBilling  PaymentMethod  MonthlyCharges TotalCharges  Churn  
0                 1              0           29.85        29.85      0  
1                 0              1           56.95       1889.5      0  
2                 1              1           53.85       108.15      1  
3                 0              2           42.30      1840.75      0  
4                 1              0           70.70       151.65      1

In [5]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

In [6]:
df.dtypes

gender                int64
SeniorCitizen         int64
Partner               int64
Dependents            int64
tenure                int64
PhoneService          int64
MultipleLines         int64
InternetService       int64
OnlineSecurity        int64
OnlineBackup          int64
DeviceProtection      int64
TechSupport           int64
StreamingTV           int64
StreamingMovies       int64
Contract              int64
PaperlessBilling      int64
PaymentMethod         int64
MonthlyCharges      float64
TotalCharges        float64
Churn                 int64
dtype: object

In [7]:
# split into input (X) and output (Y) variables
X = df.loc[:,df.columns != 'Churn']
Y = df.loc[:, df.columns == 'Churn']

In [12]:
import numpy as np
from tensorflow import keras
from keras import backend
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# Instantiate our model
model = Sequential()
# Add a Dense layer where all the nodes are connected between layers
model.add(Dense(13, input_dim=19, activation='relu'))
model.add(Dense(13, activation='relu'))
model.add(Dense(13, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Add a Compile method; which is how we want to train/teach the dataset
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# print model summary
print(model.summary())
# Fit the model using training data; use 10% of data for cross validation
model.fit(X, Y, epochs=5, validation_split=0.10)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 13)                260       
_________________________________________________________________
dense_13 (Dense)             (None, 13)                182       
_________________________________________________________________
dense_14 (Dense)             (None, 13)                182       
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 14        
Total params: 638
Trainable params: 638
Non-trainable params: 0
_________________________________________________________________
None
Train on 6338 samples, validate on 705 samples
Epoch 1/5
6338/6338 [==============================] - 1s 83us/sample - loss: nan - accuracy: 0.7210 - val_loss: nan - val_accuracy: 0.7234
Epoch 2/5
6338/6338 [==============================] - 0s 56us

In [11]:

# Function to create model, required for KerasClassifier
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=19, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
param_grid = {'batch_size': [10],
              'epochs': [5]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, Y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Best: 0.7346301119587866 using {'batch_size': 10, 'epochs': 5}


In [13]:
import wandb
from wandb.keras import WandbCallback

In [14]:
wandb.init(project="boston", entity="lambda-ds7")

# Important Hyperparameters
inputs = X.shape[1]
wandb.config.epochs = 50
wandb.config.batch_size = 10

# Create Model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(inputs,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))
# Compile Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fit Model

model.fit(X, Y, 
          validation_split=0.33, 
          epochs=wandb.config.epochs, 
          batch_size=wandb.config.epochs, 
          callbacks=[WandbCallback()]
         )

Train on 4718 samples, validate on 2325 samples
Epoch 1/50
4718/4718 [==============================] - 1s 123us/sample - loss: nan - accuracy: 0.7384 - val_loss: nan - val_accuracy: 0.7269
Epoch 2/50
4718/4718 [==============================] - 0s 49us/sample - loss: nan - accuracy: 0.7384 - val_loss: nan - val_accuracy: 0.7269
Epoch 3/50
4718/4718 [==============================] - 0s 43us/sample - loss: nan - accuracy: 0.7384 - val_loss: nan - val_accuracy: 0.7269
Epoch 4/50
4718/4718 [==============================] - 0s 45us/sample - loss: nan - accuracy: 0.7384 - val_loss: nan - val_accuracy: 0.7269
Epoch 5/50
4718/4718 [==============================] - 0s 44us/sample - loss: nan - accuracy: 0.7384 - val_loss: nan - val_accuracy: 0.7269
Epoch 6/50
4718/4718 [==============================] - 0s 45us/sample - loss: nan - accuracy: 0.7384 - val_loss: nan - val_accuracy: 0.7269
Epoch 7/50
4718/4718 [==============================] - 0s 50us/sample - loss: nan - accuracy: 0.7384 - v

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?